Test the performence of deep-ae

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [18]:
import numpy as np
import pandas as pd

import pickle
import matplotlib.pyplot as plt
%matplotlib inline

from keras.layers import Input, Dense, Dropout
from keras.models import Model
import keras

from sklearn import preprocessing
import seaborn as sns

Globals

In [3]:
early_stopping_monitor = keras.callbacks.EarlyStopping(monitor='loss', patience=20,mode='min')

Load data

In [4]:
pklfile = '/Users/bogao/DataFiles/new landscape/data/breast_bands.pkl'
with open(pklfile, 'rb') as fi:
    feat_breast = pickle.load(fi)
    
pklfile = '/Users/bogao/DataFiles/new landscape/data/ovary_bands.pkl'
with open(pklfile, 'rb') as fi:
    feat_ovary = pickle.load(fi)
    
pklfile = '/Users/bogao/DataFiles/new landscape/data/skin_bands.pkl'
with open(pklfile, 'rb') as fi:
    feat_skin = pickle.load(fi)

Scaling data to [0,1]

In [5]:
feat_breast = preprocessing.MinMaxScaler().fit_transform(np.abs(feat_breast))
feat_ovary = preprocessing.MinMaxScaler().fit_transform(np.abs(feat_ovary))
feat_skin = preprocessing.MinMaxScaler().fit_transform(np.abs(feat_skin))

In [ ]:
feat_breast.shape

In [ ]:
feat_ovary.shape

### Models

Basic Model

In [28]:
def basic_dae(feat_mat, noise_factor=0.05, hidden_size=800, batch_size=128, epochs=2000):
    
    input_size = feat_mat.shape[1]
    feat_noisy = feat_mat + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=feat_mat.shape) 
    feat_noisy = np.clip(feat_noisy, 0., 1.)

    x = Input(shape=(input_size,))
    h = Dense(hidden_size, activation='relu')(x)
    r = Dense(input_size, activation='sigmoid')(h)

    dae = Model(inputs=x, outputs=r)
    dae.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

    history_base = dae.fit(feat_noisy, feat_noisy, batch_size=batch_size,epochs=epochs, callbacks = [early_stopping_monitor],shuffle=True)
    encoder = Model(x,h)
    return dae, encoder

Deep DAE

In [29]:
def deep_dae(feat_mat, noise_factor=0.05, layers=2, start_size=1024, batch_size=128, epochs=2000):
    
    input_size = feat_mat.shape[1]
    feat_noisy = feat_mat + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=feat_mat.shape) 
    feat_noisy = np.clip(feat_noisy, 0., 1.)

    x = Input(shape=(input_size,))
    # First hidden encoder layer
    hidden_encoders = Dense(start_size, activation='relu')(x)
    
    # Adding more layers
    for i in range(1,layers):
        layer_size = int(start_size/(2**i))
        hidden_encoders = Dense(layer_size, activation='relu')(hidden_encoders)
        
    # First hidden decoder layer
    hidden_decoder = Dense(int(start_size/(2**(layers-2))), activation='relu')(hidden_encoders)
    
    # Adding more layers
    for i in range(layers-3,-1,-1):
        layer_size = int(start_size/(2**i))
        hidden_decoder = Dense(layer_size, activation='relu')(hidden_decoder)
    
    r = Dense(input_size, activation='sigmoid')(hidden_decoder)

    dae = Model(inputs=x, outputs=r)
    dae.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

    history_base = dae.fit(feat_noisy, feat_noisy, batch_size=batch_size,epochs=epochs, callbacks = [early_stopping_monitor],shuffle=True)
    encoder = Model(x,hidden_encoders)
    return dae, encoder

### Compare depth

Settings

In [ ]:
def depth_test(feat_mat):
    
    models = []
    labels = ['basic', 'layers_2', 'layers_3', 'layers_4', 'layers_5', 'layers_4_exp', 'layers_5_exp']
    
    models.append(basic_dae(feat_mat))
    models.append(deep_dae(feat_mat, layers=2))
    models.append(deep_dae(feat_mat, layers=3))
    models.append(deep_dae(feat_mat, layers=4))
    models.append(deep_dae(feat_mat, layers=5))
    models.append(deep_dae(feat_mat, layers=4, start_size=2048))
    models.append(deep_dae(feat_mat, layers=5, start_size=2048))
    
    return models, labels

Plot function

In [ ]:
def plotHis(his_list, labels, loss_y_top=0):
    plt.figure(figsize=(40,20))

    plt.subplot(1,2, 1)
    for his in his_list:
        plt.plot(his.history['loss'])
    plt.title('Models loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    if loss_y_top > 0:
        plt.ylim(top=loss_y_top)
    plt.legend(labels, loc='upper right')

    plt.subplot(1,2, 2)
    for his in his_list:
        plt.plot(his.history['acc'])
    plt.title('Models accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(labels, loc='upper right')

    plt.show()

Breast

In [ ]:
models_breast, labels_breast = depth_test(feat_breast)

In [ ]:
his_breast = [x[0].history for x in models_breast]
plotHis(his_breast, labels_breast)

Ovary

In [ ]:
models_ovary, labels_ovary = depth_test(feat_ovary)

In [ ]:
his_ovary = [x[0].history for x in models_ovary]
plotHis(his_ovary, labels_ovary)

Skin

In [ ]:
models_skin, labels_skin = depth_test(feat_skin)

In [ ]:
his_skin = [x[0].history for x in models_skin]
plotHis(his_skin, labels_skin)

In [10]:
model = deep_dae(feat_skin, epochs=20)

Epoch 1/20
1238/1238 [==============================] - 2s 1ms/step - loss: 0.0673 - acc: 8.0775e-04
Epoch 2/20
1238/1238 [==============================] - 1s 487us/step - loss: 0.0232 - acc: 0.0000e+00
Epoch 3/20
1238/1238 [==============================] - 1s 483us/step - loss: 0.0234 - acc: 0.0024
Epoch 4/20
1238/1238 [==============================] - 1s 474us/step - loss: 0.0233 - acc: 0.0024
Epoch 5/20
1238/1238 [==============================] - 1s 483us/step - loss: 0.0230 - acc: 0.0024
Epoch 6/20
1238/1238 [==============================] - 1s 483us/step - loss: 0.0228 - acc: 0.0097
Epoch 7/20
1238/1238 [==============================] - 1s 487us/step - loss: 0.0225 - acc: 0.0040
Epoch 8/20
1238/1238 [==============================] - 1s 483us/step - loss: 0.0223 - acc: 0.0024
Epoch 9/20
1238/1238 [==============================] - 1s 485us/step - loss: 0.0220 - acc: 8.0775e-04
Epoch 10/20
1238/1238 [==============================] - 1s 483us/step - loss: 0.0218 - acc: 0.0048

In [66]:
noise_factor=0.05
feat_mat = feat_skin[:,:300]
input_size = feat_mat.shape[1]
feat_noisy = feat_mat + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=feat_mat.shape) 
feat_noisy = np.clip(feat_noisy, 0., 1.)

x = Input(shape=(input_size,))
h1 = Dense(256, activation='relu')(x)
# d1 = Dropout(0.5)(h1)
h = Dense(128, activation='relu')(h1)
h2 = Dense(256, activation='relu')(h)
# d2 = Dropout(0.5)(h2)
r = Dense(input_size, activation='sigmoid')(h2)

dae = Model(inputs=x, outputs=r)
dae.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

history_base = dae.fit(feat_noisy, feat_noisy, batch_size=128,epochs=2000)

Epoch 1/2000
1238/1238 [==============================] - 2s 2ms/step - loss: 0.1742 - acc: 0.0000e+00
Epoch 2/2000
1238/1238 [==============================] - 0s 91us/step - loss: 0.0462 - acc: 0.0000e+00
Epoch 3/2000
1238/1238 [==============================] - 0s 89us/step - loss: 0.0177 - acc: 0.0000e+00
Epoch 4/2000
1238/1238 [==============================] - 0s 89us/step - loss: 0.0177 - acc: 0.0000e+00
Epoch 5/2000
1238/1238 [==============================] - 0s 88us/step - loss: 0.0177 - acc: 0.0000e+00
Epoch 6/2000
1238/1238 [==============================] - 0s 91us/step - loss: 0.0178 - acc: 0.0000e+00
Epoch 7/2000
1238/1238 [==============================] - 0s 89us/step - loss: 0.0178 - acc: 0.0000e+00
Epoch 8/2000
1238/1238 [==============================] - 0s 90us/step - loss: 0.0178 - acc: 0.0000e+00
Epoch 9/2000
1238/1238 [==============================] - 0s 88us/step - loss: 0.0178 - acc: 0.0000e+00
Epoch 10/2000
1238/1238 [==============================] - 0s 89u

1238/1238 [==============================] - 0s 89us/step - loss: 0.0031 - acc: 0.1438
Epoch 161/2000
1238/1238 [==============================] - 0s 88us/step - loss: 0.0031 - acc: 0.1430
Epoch 162/2000
1238/1238 [==============================] - 0s 87us/step - loss: 0.0031 - acc: 0.1422
Epoch 163/2000
1238/1238 [==============================] - 0s 90us/step - loss: 0.0030 - acc: 0.1462
Epoch 164/2000
1238/1238 [==============================] - 0s 87us/step - loss: 0.0030 - acc: 0.1397
Epoch 165/2000
1238/1238 [==============================] - 0s 89us/step - loss: 0.0030 - acc: 0.1397
Epoch 166/2000
1238/1238 [==============================] - 0s 89us/step - loss: 0.0029 - acc: 0.1575
Epoch 167/2000
1238/1238 [==============================] - 0s 89us/step - loss: 0.0029 - acc: 0.1575
Epoch 168/2000
1238/1238 [==============================] - 0s 89us/step - loss: 0.0029 - acc: 0.1624
Epoch 169/2000
1238/1238 [==============================] - 0s 88us/step - loss: 0.0028 - acc: 0.

1238/1238 [==============================] - 0s 88us/step - loss: 0.0014 - acc: 0.4402
Epoch 321/2000
1238/1238 [==============================] - 0s 88us/step - loss: 0.0014 - acc: 0.4435
Epoch 322/2000
1238/1238 [==============================] - 0s 87us/step - loss: 0.0014 - acc: 0.4459
Epoch 323/2000
1238/1238 [==============================] - 0s 87us/step - loss: 0.0014 - acc: 0.4564
Epoch 324/2000
1238/1238 [==============================] - 0s 91us/step - loss: 0.0014 - acc: 0.4507
Epoch 325/2000
1238/1238 [==============================] - 0s 88us/step - loss: 0.0015 - acc: 0.4378
Epoch 326/2000
1238/1238 [==============================] - 0s 88us/step - loss: 0.0017 - acc: 0.4281
Epoch 327/2000
1238/1238 [==============================] - 0s 88us/step - loss: 0.0016 - acc: 0.4321
Epoch 328/2000
1238/1238 [==============================] - 0s 89us/step - loss: 0.0015 - acc: 0.4540
Epoch 329/2000
1238/1238 [==============================] - 0s 87us/step - loss: 0.0015 - acc: 0.

1238/1238 [==============================] - 0s 90us/step - loss: 0.0011 - acc: 0.5711
Epoch 481/2000
1238/1238 [==============================] - 0s 89us/step - loss: 0.0011 - acc: 0.5606
Epoch 482/2000
1238/1238 [==============================] - 0s 90us/step - loss: 0.0011 - acc: 0.5808
Epoch 483/2000
1238/1238 [==============================] - 0s 90us/step - loss: 0.0011 - acc: 0.5687
Epoch 484/2000
1238/1238 [==============================] - 0s 88us/step - loss: 0.0011 - acc: 0.5719
Epoch 485/2000
1238/1238 [==============================] - 0s 88us/step - loss: 0.0011 - acc: 0.5856
Epoch 486/2000
1238/1238 [==============================] - 0s 92us/step - loss: 0.0011 - acc: 0.5897
Epoch 487/2000
1238/1238 [==============================] - 0s 94us/step - loss: 0.0011 - acc: 0.5719
Epoch 488/2000
1238/1238 [==============================] - 0s 89us/step - loss: 0.0011 - acc: 0.5703
Epoch 489/2000
1238/1238 [==============================] - 0s 89us/step - loss: 0.0011 - acc: 0.

1238/1238 [==============================] - 0s 90us/step - loss: 9.0946e-04 - acc: 0.6365
Epoch 637/2000
1238/1238 [==============================] - 0s 89us/step - loss: 9.1084e-04 - acc: 0.6349
Epoch 638/2000
1238/1238 [==============================] - 0s 88us/step - loss: 9.0376e-04 - acc: 0.6422
Epoch 639/2000
1238/1238 [==============================] - 0s 88us/step - loss: 9.0136e-04 - acc: 0.6341
Epoch 640/2000
1238/1238 [==============================] - 0s 88us/step - loss: 8.9961e-04 - acc: 0.6349
Epoch 641/2000
1238/1238 [==============================] - 0s 90us/step - loss: 9.0212e-04 - acc: 0.6462
Epoch 642/2000
1238/1238 [==============================] - 0s 87us/step - loss: 9.0347e-04 - acc: 0.6470
Epoch 643/2000
1238/1238 [==============================] - 0s 89us/step - loss: 9.0287e-04 - acc: 0.6478
Epoch 644/2000
1238/1238 [==============================] - 0s 89us/step - loss: 9.0537e-04 - acc: 0.6494
Epoch 645/2000
1238/1238 [==============================] - 0

...

1238/1238 [==============================] - 0s 90us/step - loss: 6.1511e-04 - acc: 0.7391
Epoch 1250/2000
1238/1238 [==============================] - 0s 88us/step - loss: 6.1752e-04 - acc: 0.7536
Epoch 1251/2000
1238/1238 [==============================] - 0s 89us/step - loss: 6.1783e-04 - acc: 0.7318
Epoch 1252/2000
1238/1238 [==============================] - 0s 88us/step - loss: 6.2044e-04 - acc: 0.7431
Epoch 1253/2000
1238/1238 [==============================] - 0s 90us/step - loss: 6.1939e-04 - acc: 0.7496
Epoch 1254/2000
1238/1238 [==============================] - 0s 86us/step - loss: 6.1489e-04 - acc: 0.7367
Epoch 1255/2000
1238/1238 [==============================] - 0s 89us/step - loss: 6.1297e-04 - acc: 0.7447
Epoch 1256/2000
1238/1238 [==============================] - 0s 87us/step - loss: 6.1264e-04 - acc: 0.7262
Epoch 1257/2000
1238/1238 [==============================] - 0s 88us/step - loss: 6.1879e-04 - acc: 0.7326
Epoch 1258/2000
1238/1238 [==========================

1238/1238 [==============================] - 0s 90us/step - loss: 5.9515e-04 - acc: 0.7367
Epoch 1402/2000
1238/1238 [==============================] - 0s 88us/step - loss: 5.9517e-04 - acc: 0.7423
Epoch 1403/2000
1238/1238 [==============================] - 0s 87us/step - loss: 6.0232e-04 - acc: 0.7326
Epoch 1404/2000
1238/1238 [==============================] - 0s 87us/step - loss: 6.0357e-04 - acc: 0.7439
Epoch 1405/2000
1238/1238 [==============================] - 0s 92us/step - loss: 6.0584e-04 - acc: 0.7423
Epoch 1406/2000
1238/1238 [==============================] - 0s 89us/step - loss: 5.9724e-04 - acc: 0.7334
Epoch 1407/2000
1238/1238 [==============================] - 0s 90us/step - loss: 5.9189e-04 - acc: 0.7447
Epoch 1408/2000
1238/1238 [==============================] - 0s 89us/step - loss: 5.8414e-04 - acc: 0.7367
Epoch 1409/2000
1238/1238 [==============================] - 0s 89us/step - loss: 5.7644e-04 - acc: 0.7601
Epoch 1410/2000
1238/1238 [==========================

1238/1238 [==============================] - 0s 90us/step - loss: 5.5934e-04 - acc: 0.7456
Epoch 1554/2000
1238/1238 [==============================] - 0s 90us/step - loss: 5.6757e-04 - acc: 0.7512
Epoch 1555/2000
1238/1238 [==============================] - 0s 89us/step - loss: 5.7067e-04 - acc: 0.7528
Epoch 1556/2000
1238/1238 [==============================] - 0s 92us/step - loss: 5.6984e-04 - acc: 0.7423
Epoch 1557/2000
1238/1238 [==============================] - 0s 88us/step - loss: 5.6621e-04 - acc: 0.7456
Epoch 1558/2000
1238/1238 [==============================] - 0s 90us/step - loss: 5.6250e-04 - acc: 0.7512
Epoch 1559/2000
1238/1238 [==============================] - 0s 90us/step - loss: 5.6416e-04 - acc: 0.7520
Epoch 1560/2000
1238/1238 [==============================] - 0s 89us/step - loss: 5.6229e-04 - acc: 0.7334
Epoch 1561/2000
1238/1238 [==============================] - 0s 89us/step - loss: 5.5811e-04 - acc: 0.7415
Epoch 1562/2000
1238/1238 [==========================

1238/1238 [==============================] - 0s 92us/step - loss: 5.3278e-04 - acc: 0.7609
Epoch 1706/2000
1238/1238 [==============================] - 0s 87us/step - loss: 5.2967e-04 - acc: 0.7601
Epoch 1707/2000
1238/1238 [==============================] - 0s 88us/step - loss: 5.2641e-04 - acc: 0.7593
Epoch 1708/2000
1238/1238 [==============================] - 0s 88us/step - loss: 5.2693e-04 - acc: 0.7649
Epoch 1709/2000
1238/1238 [==============================] - 0s 88us/step - loss: 5.2614e-04 - acc: 0.7658
Epoch 1710/2000
1238/1238 [==============================] - 0s 90us/step - loss: 5.2675e-04 - acc: 0.7569
Epoch 1711/2000
1238/1238 [==============================] - 0s 87us/step - loss: 5.2736e-04 - acc: 0.7423
Epoch 1712/2000
1238/1238 [==============================] - 0s 88us/step - loss: 5.2768e-04 - acc: 0.7641
Epoch 1713/2000
1238/1238 [==============================] - 0s 88us/step - loss: 5.2369e-04 - acc: 0.7609
Epoch 1714/2000
1238/1238 [==========================

1238/1238 [==============================] - 0s 95us/step - loss: 5.1028e-04 - acc: 0.7771
Epoch 1858/2000
1238/1238 [==============================] - 0s 92us/step - loss: 5.1048e-04 - acc: 0.7633
Epoch 1859/2000
1238/1238 [==============================] - 0s 91us/step - loss: 5.0985e-04 - acc: 0.7561
Epoch 1860/2000
1238/1238 [==============================] - 0s 91us/step - loss: 5.0857e-04 - acc: 0.7698
Epoch 1861/2000
1238/1238 [==============================] - 0s 90us/step - loss: 5.0825e-04 - acc: 0.7763
Epoch 1862/2000
1238/1238 [==============================] - 0s 88us/step - loss: 5.0879e-04 - acc: 0.7593
Epoch 1863/2000
1238/1238 [==============================] - 0s 90us/step - loss: 5.1006e-04 - acc: 0.7641
Epoch 1864/2000
1238/1238 [==============================] - 0s 89us/step - loss: 5.0595e-04 - acc: 0.7787
Epoch 1865/2000
1238/1238 [==============================] - 0s 92us/step - loss: 5.0661e-04 - acc: 0.7633
Epoch 1866/2000
1238/1238 [==========================

In [51]:
dae.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        (None, 1622)              0         
_________________________________________________________________
dense_109 (Dense)            (None, 128)               207744    
_________________________________________________________________
dense_110 (Dense)            (None, 64)                8256      
_________________________________________________________________
dense_111 (Dense)            (None, 128)               8320      
_________________________________________________________________
dense_112 (Dense)            (None, 1622)              209238    
Total params: 433,558
Trainable params: 433,558
Non-trainable params: 0
_________________________________________________________________


In [67]:
model, encoder = basic_dae(feat_skin[:,:300], epochs=2000)

Epoch 1/2000
1238/1238 [==============================] - 2s 2ms/step - loss: 0.1569 - acc: 0.0024
Epoch 2/2000
1238/1238 [==============================] - 0s 97us/step - loss: 0.0451 - acc: 0.0065
Epoch 3/2000
1238/1238 [==============================] - 0s 99us/step - loss: 0.0194 - acc: 0.0057
Epoch 4/2000
1238/1238 [==============================] - 0s 99us/step - loss: 0.0177 - acc: 0.0145
Epoch 5/2000
1238/1238 [==============================] - 0s 100us/step - loss: 0.0176 - acc: 0.0186
Epoch 6/2000
1238/1238 [==============================] - 0s 103us/step - loss: 0.0176 - acc: 0.0186
Epoch 7/2000
1238/1238 [==============================] - 0s 105us/step - loss: 0.0175 - acc: 0.0186
Epoch 8/2000
1238/1238 [==============================] - 0s 99us/step - loss: 0.0175 - acc: 0.0258
Epoch 9/2000
1238/1238 [==============================] - 0s 97us/step - loss: 0.0175 - acc: 0.0218
Epoch 10/2000
1238/1238 [==============================] - 0s 97us/step - loss: 0.0175 - acc: 0.02

1238/1238 [==============================] - 0s 118us/step - loss: 0.0013 - acc: 0.5048
Epoch 161/2000
1238/1238 [==============================] - 0s 112us/step - loss: 0.0012 - acc: 0.5040
Epoch 162/2000
1238/1238 [==============================] - 0s 105us/step - loss: 0.0012 - acc: 0.5202
Epoch 163/2000
1238/1238 [==============================] - 0s 106us/step - loss: 0.0012 - acc: 0.5121
Epoch 164/2000
1238/1238 [==============================] - 0s 105us/step - loss: 0.0012 - acc: 0.5113
Epoch 165/2000
1238/1238 [==============================] - 0s 105us/step - loss: 0.0012 - acc: 0.5162
Epoch 166/2000
1238/1238 [==============================] - 0s 110us/step - loss: 0.0012 - acc: 0.5145
Epoch 167/2000
1238/1238 [==============================] - 0s 112us/step - loss: 0.0012 - acc: 0.5186
Epoch 168/2000
1238/1238 [==============================] - 0s 111us/step - loss: 0.0012 - acc: 0.5267
Epoch 169/2000
1238/1238 [==============================] - 0s 114us/step - loss: 0.0012

1238/1238 [==============================] - 0s 106us/step - loss: 7.0467e-04 - acc: 0.7108
Epoch 315/2000
1238/1238 [==============================] - 0s 104us/step - loss: 6.9874e-04 - acc: 0.7116
Epoch 316/2000
1238/1238 [==============================] - 0s 104us/step - loss: 6.9637e-04 - acc: 0.7165
Epoch 317/2000
1238/1238 [==============================] - 0s 105us/step - loss: 7.0361e-04 - acc: 0.7124
Epoch 318/2000
1238/1238 [==============================] - 0s 105us/step - loss: 7.1817e-04 - acc: 0.7132
Epoch 319/2000
1238/1238 [==============================] - 0s 104us/step - loss: 7.2060e-04 - acc: 0.7100
Epoch 320/2000
1238/1238 [==============================] - 0s 106us/step - loss: 7.4948e-04 - acc: 0.7181
Epoch 321/2000
1238/1238 [==============================] - 0s 106us/step - loss: 7.3467e-04 - acc: 0.7270
Epoch 322/2000
1238/1238 [==============================] - 0s 106us/step - loss: 7.3879e-04 - acc: 0.7084
Epoch 323/2000
1238/1238 [==========================

In [53]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        (None, 1622)              0         
_________________________________________________________________
dense_113 (Dense)            (None, 800)               1298400   
_________________________________________________________________
dense_114 (Dense)            (None, 1622)              1299222   
Total params: 2,597,622
Trainable params: 2,597,622
Non-trainable params: 0
_________________________________________________________________


with open('/Users/bogao/DataFiles/new landscape/data/breast_bands.pkl', 'rb') as fi:
    feat_sob = pickle.load(fi)

In [33]:
feat_sob = preprocessing.MinMaxScaler().fit_transform(np.abs(feat_sob))

In [37]:
model, encoder = basic_dae(feat_skin, epochs=100)

Epoch 1/100
1238/1238 [==============================] - 2s 1ms/step - loss: 0.1124 - acc: 0.0040
Epoch 2/100
1238/1238 [==============================] - 0s 309us/step - loss: 0.0174 - acc: 0.0065
Epoch 3/100
1238/1238 [==============================] - 0s 301us/step - loss: 0.0170 - acc: 0.0073
Epoch 4/100
1238/1238 [==============================] - 0s 297us/step - loss: 0.0171 - acc: 0.0040
Epoch 5/100
1238/1238 [==============================] - 0s 296us/step - loss: 0.0171 - acc: 0.0040
Epoch 6/100
1238/1238 [==============================] - 0s 299us/step - loss: 0.0171 - acc: 0.0040
Epoch 7/100
1238/1238 [==============================] - 0s 299us/step - loss: 0.0171 - acc: 0.0040
Epoch 8/100
1238/1238 [==============================] - 0s 304us/step - loss: 0.0171 - acc: 0.0040
Epoch 9/100
1238/1238 [==============================] - 0s 307us/step - loss: 0.0171 - acc: 0.0040
Epoch 10/100
1238/1238 [==============================] - 0s 302us/step - loss: 0.0171 - acc: 0.0040
E

In [43]:
mnist = keras.datasets.mnist
(X_train, _), (X_test, _) = mnist.load_data()

In [44]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
X_train = X_train.reshape((len(X_train), np.prod(X_train.shape[1:])))
X_test = X_test.reshape((len(X_test), np.prod(X_test.shape[1:])))

In [62]:
X_train[:600,].shape

(600, 784)

In [61]:
feat_skin.shape

(1238, 1622)

In [69]:
1622*801

1299222

In [71]:
88/60

1.4666666666666666